In [1]:
import pandas as pd
import numpy as np
import scipy.sparse as sps
import random as rnd

from scipy.sparse import *

from matplotlib import pyplot

In [87]:
urm_path = './content/data_train.csv'
urm_all_df = pd.read_csv(filepath_or_buffer=urm_path,
                                sep=",",
                                header=0,
                                dtype={0:int, 1:int, 2:float},
                                engine='python')

urm_all_df.columns = ["UserID", "ItemID", "Interaction"]

In [3]:
urm_all_df.head(10)

,UserID,ItemID,Interaction
0,1,7,1.0
1,1,15,1.0
2,1,16,1.0
3,1,133,1.0
4,1,161,1.0
5,1,187,1.0
6,1,205,1.0
7,1,222,1.0
8,1,237,1.0
9,1,354,1.0


In [4]:
print ("The number of interactions is {}".format(len(urm_all_df)))

The number of interactions is 478730


In [5]:
userID_unique = urm_all_df["UserID"].unique()
itemID_unique = urm_all_df["ItemID"].unique()

n_users = len(userID_unique)
n_items = len(itemID_unique)
n_interactions = len(urm_all_df)

print ("Number of items\t {}, Number of users\t {}".format(n_items, n_users))
print ("Max ID items\t {}, Max Id users\t {}\n".format(max(itemID_unique), max(userID_unique)))
print ("Average interactions per user {:.2f}".format(n_interactions/n_users))
print ("Average interactions per item {:.2f}\n".format(n_interactions/n_items))

print ("Sparsity {:.2f} %".format((1-float(n_interactions)/(n_items*n_users))*100))

Number of items	 22222, Number of users	 12638
Max ID items	 22347, Max Id users	 13024

Average interactions per user 37.88
Average interactions per item 21.54

Sparsity 99.83 %


# Remove empty profiles

In [6]:
# Remove empty IDs
mapped_id, original_id = pd.factorize(urm_all_df["UserID"].unique())
user_original_ID_to_index = pd.Series(mapped_id, index=original_id)

mapped_id, original_id = pd.factorize(urm_all_df["ItemID"].unique())
item_original_ID_to_index = pd.Series(mapped_id, index=original_id)



# Let's create the mapping from items indices to original item_ids
item_index_to_original_ID = pd.Series(item_original_ID_to_index.index,index = item_original_ID_to_index.values)



original_item_ID = 125
print("New index for item {} is {}".format(original_item_ID, item_original_ID_to_index[original_item_ID]))


urm_all_df["UserID"] = urm_all_df["UserID"].map(user_original_ID_to_index)
urm_all_df["ItemID"] = urm_all_df["ItemID"].map(item_original_ID_to_index)
urm_all_df.head(n=10)

New index for item 125 is 93


,UserID,ItemID,Interaction
0,0,0,1.0
1,0,1,1.0
2,0,2,1.0
3,0,3,1.0
4,0,4,1.0
5,0,5,1.0
6,0,6,1.0
7,0,7,1.0
8,0,8,1.0
9,0,9,1.0


In [7]:
userID_unique = urm_all_df["UserID"].unique()
itemID_unique = urm_all_df["ItemID"].unique()

n_users = len(userID_unique)
n_items = len(itemID_unique)
n_interactions = len(urm_all_df)

print ("Number of items\t {}, Number of users\t {}".format(n_items, n_users))
print ("Max ID items\t {}, Max Id users\t {}\n".format(max(itemID_unique), max(userID_unique)))
print ("Average interactions per user {:.2f}".format(n_interactions/n_users))
print ("Average interactions per item {:.2f}\n".format(n_interactions/n_items))

print ("Sparsity {:.2f} %".format((1-float(n_interactions)/(n_items*n_users))*100))

Number of items	 22222, Number of users	 12638
Max ID items	 22221, Max Id users	 12637

Average interactions per user 37.88
Average interactions per item 21.54

Sparsity 99.83 %


In [88]:
urm_all = sps.coo_matrix((urm_all_df["Interaction"].values,
                          (urm_all_df["UserID"].values, urm_all_df["ItemID"].values)))

urm_all

<13025x22348 sparse matrix of type '<class 'numpy.float64'>'
	with 478730 stored elements in COOrdinate format>

In [89]:
urm_all = urm_all.tocsr()
urm_all

<13025x22348 sparse matrix of type '<class 'numpy.float64'>'
	with 478730 stored elements in Compressed Sparse Row format>

# K-fold Cross-Validatoin


### Load the similarity matrices of models trained on different train folds

In [10]:
from scipy.sparse import save_npz, load_npz

In [11]:
data_folds = []

for fold_n in range(5):
  fold_urm_train = load_npz(f'./5-fold CV/Folds/train_noemptyprofiles_fold{fold_n}.npz')
  fold_urm_val = load_npz(f'./5-fold CV/Folds/val_noemptyprofiles_fold{fold_n}.npz')
  data_folds.append({"train":fold_urm_train,"val":fold_urm_val})

print(data_folds)

[{'train': <12638x22222 sparse matrix of type '<class 'numpy.float64'>'
	with 382984 stored elements in Compressed Sparse Row format>, 'val': <12638x22217 sparse matrix of type '<class 'numpy.float64'>'
	with 95746 stored elements in Compressed Sparse Row format>}, {'train': <12638x22221 sparse matrix of type '<class 'numpy.float64'>'
	with 382984 stored elements in Compressed Sparse Row format>, 'val': <12638x22222 sparse matrix of type '<class 'numpy.float64'>'
	with 95746 stored elements in Compressed Sparse Row format>}, {'train': <12638x22222 sparse matrix of type '<class 'numpy.float64'>'
	with 382984 stored elements in Compressed Sparse Row format>, 'val': <12638x22218 sparse matrix of type '<class 'numpy.float64'>'
	with 95746 stored elements in Compressed Sparse Row format>}, {'train': <12638x22222 sparse matrix of type '<class 'numpy.float64'>'
	with 382984 stored elements in Compressed Sparse Row format>, 'val': <12638x22220 sparse matrix of type '<class 'numpy.float64'>'
	w

In [12]:
# SLIM elasticnet

S_slim_fold0 = load_npz('./5-fold CV/SLIMelasticnet/ItItSimilarity_trained_fold0_noemptyprofiles_SLIMelasticnet_kaggleMAP0.13484.npz')

S_slim_fold1 = load_npz('./5-fold CV/SLIMelasticnet/ItItSimilarity_trained_fold1_noemptyprofiles_SLIMelasticnet_kaggleMAP0.13484.npz')

S_slim_fold2 = load_npz('./5-fold CV/SLIMelasticnet/ItItSimilarity_trained_fold2_noemptyprofiles_SLIMelasticnet_kaggleMAP0.13484.npz')

S_slim_fold3 = load_npz('./5-fold CV/SLIMelasticnet/ItItSimilarity_trained_fold3_noemptyprofiles_SLIMelasticnet_kaggleMAP0.13484.npz')

S_slim_fold4 = load_npz('./5-fold CV/SLIMelasticnet/ItItSimilarity_trained_fold4_noemptyprofiles_SLIMelasticnet_kaggleMAP0.13484.npz')

In [13]:
S_slim_fold0.indices.shape

(2240133,)

In [14]:
S_slim_fold0.data.shape

(2240133,)

In [15]:
# IBCF

S_ibcf_fold0 = load_npz('./5-fold CV/IBCF/ItItSimilarity_trained_fold0_noemptyprofiles_IBCF_kaggleMAP0.11585.npz')

S_ibcf_fold1 = load_npz('./5-fold CV/IBCF/ItItSimilarity_trained_fold1_noemptyprofiles_IBCF_kaggleMAP0.11585.npz')

S_ibcf_fold2 = load_npz('./5-fold CV/IBCF/ItItSimilarity_trained_fold2_noemptyprofiles_IBCF_kaggleMAP0.11585.npz')

S_ibcf_fold3 = load_npz('./5-fold CV/IBCF/ItItSimilarity_trained_fold3_noemptyprofiles_IBCF_kaggleMAP0.11585.npz')

S_ibcf_fold4 = load_npz('./5-fold CV/IBCF/ItItSimilarity_trained_fold4_noemptyprofiles_IBCF_kaggleMAP0.11585.npz')

In [16]:
S_ibcf_fold0.indices.shape

(308988,)

In [17]:
S_ibcf_fold0.data.shape

(308988,)

In [18]:
# RP3Beta

S_rp3beta_fold0 = load_npz('./5-fold CV/RP3Beta/ItItSimilarity_trained_fold0_noemptyprofiles_RP3Beta_kaggleMAP0.09372.npz')

S_rp3beta_fold1 = load_npz('./5-fold CV/RP3Beta/ItItSimilarity_trained_fold1_noemptyprofiles_RP3Beta_kaggleMAP0.09372.npz')

S_rp3beta_fold2 = load_npz('./5-fold CV/RP3Beta/ItItSimilarity_trained_fold2_noemptyprofiles_RP3Beta_kaggleMAP0.09372.npz')

S_rp3beta_fold3 = load_npz('./5-fold CV/RP3Beta/ItItSimilarity_trained_fold3_noemptyprofiles_RP3Beta_kaggleMAP0.09372.npz')

S_rp3beta_fold4 = load_npz('./5-fold CV/RP3Beta/ItItSimilarity_trained_fold4_noemptyprofiles_RP3Beta_kaggleMAP0.09372.npz')

In [19]:
S_rp3beta_fold0.indices.shape

(3527025,)

In [20]:
S_rp3beta_fold0.data.shape

(3527025,)

In [21]:
# EASER

S_easer_fold0 = load_npz('./5-fold CV/EASER/ItItSimilarity_trained_fold0_noemptyprofiles_EASER_kaggleMAP0.12504.npz')

S_easer_fold1 = load_npz('./5-fold CV/EASER/ItItSimilarity_trained_fold1_noemptyprofiles_EASER_kaggleMAP0.12504.npz')

S_easer_fold2 = load_npz('./5-fold CV/EASER/ItItSimilarity_trained_fold2_noemptyprofiles_EASER_kaggleMAP0.12504.npz')

S_easer_fold3 = load_npz('./5-fold CV/EASER/ItItSimilarity_trained_fold3_noemptyprofiles_EASER_kaggleMAP0.12504.npz')

S_easer_fold4 = load_npz('./5-fold CV/EASER/ItItSimilarity_trained_fold4_noemptyprofiles_EASER_kaggleMAP0.12504.npz')

In [22]:
S_easer_fold0.indices.shape

(5016019,)

In [23]:
S_easer_fold0.data.shape

(5016019,)

In [24]:
S_models_fold0 = {
    "SLIM":S_slim_fold0, "IBCF":S_ibcf_fold0,"rp3beta":S_rp3beta_fold0,"EASER":S_easer_fold0
}

S_models_fold1 = {
    "SLIM":S_slim_fold1, "IBCF":S_ibcf_fold1,"rp3beta":S_rp3beta_fold1,"EASER":S_easer_fold1
}

S_models_fold2 = {
    "SLIM":S_slim_fold2, "IBCF":S_ibcf_fold2,"rp3beta":S_rp3beta_fold2,"EASER":S_easer_fold2
}

S_models_fold3 = {
    "SLIM":S_slim_fold3, "IBCF":S_ibcf_fold3,"rp3beta":S_rp3beta_fold3,"EASER":S_easer_fold3
}

S_models_fold4 = {
    "SLIM":S_slim_fold4, "IBCF":S_ibcf_fold4,"rp3beta":S_rp3beta_fold4,"EASER":S_easer_fold4
}

fold_models = [S_models_fold0,S_models_fold1,S_models_fold2,S_models_fold3,S_models_fold4]

### Load the data and similarity matrices of models trained on different RECOMMENDER Train XGboost subfolds

In [25]:
data_xgboostfolds = []

for fold_n in range(5):
  fold_urm_train_recommender = load_npz(f'./5-fold CV/XGboost_folds/Folds/recommender_train_noemptyprofiles_fold{fold_n}.npz')
  fold_urm_train_xgboost = load_npz(f'./5-fold CV/XGboost_folds/Folds/xgboost_train_noemptyprofiles_fold{fold_n}.npz')
  fold_urm_val = load_npz(f'./5-fold CV/XGboost_folds/Folds/val_noemptyprofiles_fold{fold_n}.npz')
  data_xgboostfolds.append({"train_recommender":fold_urm_train_recommender,"train_xgboost":fold_urm_train_xgboost,"val":fold_urm_val})

print(data_xgboostfolds)

[{'train_recommender': <12638x22222 sparse matrix of type '<class 'numpy.float64'>'
	with 306224 stored elements in Compressed Sparse Row format>, 'train_xgboost': <12638x22218 sparse matrix of type '<class 'numpy.float64'>'
	with 76760 stored elements in Compressed Sparse Row format>, 'val': <12638x22217 sparse matrix of type '<class 'numpy.float64'>'
	with 95746 stored elements in Compressed Sparse Row format>}, {'train_recommender': <12638x22221 sparse matrix of type '<class 'numpy.float64'>'
	with 306642 stored elements in Compressed Sparse Row format>, 'train_xgboost': <12638x22218 sparse matrix of type '<class 'numpy.float64'>'
	with 76342 stored elements in Compressed Sparse Row format>, 'val': <12638x22222 sparse matrix of type '<class 'numpy.float64'>'
	with 95746 stored elements in Compressed Sparse Row format>}, {'train_recommender': <12638x22221 sparse matrix of type '<class 'numpy.float64'>'
	with 306156 stored elements in Compressed Sparse Row format>, 'train_xgboost': <1

In [26]:
# SLIM elasticnet

S_slim_xgboostfold0 = load_npz('./5-fold CV/XGboost_folds/SLIMelasticnet/ItItSimilarity_trained_RecommenderFold0_noemptyprofiles_SLIMelasticnet_kaggleMAP0.13484.npz')

S_slim_xgboostfold1 = load_npz('./5-fold CV/XGboost_folds/SLIMelasticnet/ItItSimilarity_trained_RecommenderFold1_noemptyprofiles_SLIMelasticnet_kaggleMAP0.13484.npz')

S_slim_xgboostfold2 = load_npz('./5-fold CV/XGboost_folds/SLIMelasticnet/ItItSimilarity_trained_RecommenderFold2_noemptyprofiles_SLIMelasticnet_kaggleMAP0.13484.npz')

S_slim_xgboostfold3 = load_npz('./5-fold CV/XGboost_folds/SLIMelasticnet/ItItSimilarity_trained_RecommenderFold3_noemptyprofiles_SLIMelasticnet_kaggleMAP0.13484.npz')

S_slim_xgboostfold4 = load_npz('./5-fold CV/XGboost_folds/SLIMelasticnet/ItItSimilarity_trained_RecommenderFold4_noemptyprofiles_SLIMelasticnet_kaggleMAP0.13484.npz')

In [27]:
# IBCF

S_ibcf_xgboostfold0 = load_npz('./5-fold CV/XGboost_folds/IBCF/ItItSimilarity_trained_RecommenderFold0_noemptyprofiles_IBCF_kaggleMAP0.11585.npz')

S_ibcf_xgboostfold1 = load_npz('./5-fold CV/XGboost_folds/IBCF/ItItSimilarity_trained_RecommenderFold1_noemptyprofiles_IBCF_kaggleMAP0.11585.npz')

S_ibcf_xgboostfold2 = load_npz('./5-fold CV/XGboost_folds/IBCF/ItItSimilarity_trained_RecommenderFold2_noemptyprofiles_IBCF_kaggleMAP0.11585.npz')

S_ibcf_xgboostfold3 = load_npz('./5-fold CV/XGboost_folds/IBCF/ItItSimilarity_trained_RecommenderFold3_noemptyprofiles_IBCF_kaggleMAP0.11585.npz')

S_ibcf_xgboostfold4 = load_npz('./5-fold CV/XGboost_folds/IBCF/ItItSimilarity_trained_RecommenderFold4_noemptyprofiles_IBCF_kaggleMAP0.11585.npz')

In [28]:
# RP3Beta

S_rp3beta_xgboostfold0 = load_npz('./5-fold CV/XGboost_folds/RP3Beta/ItItSimilarity_trained_RecommenderFold0_noemptyprofiles_RP3Beta_kaggleMAP0.09372.npz')

S_rp3beta_xgboostfold1 = load_npz('./5-fold CV/XGboost_folds/RP3Beta/ItItSimilarity_trained_RecommenderFold1_noemptyprofiles_RP3Beta_kaggleMAP0.09372.npz')

S_rp3beta_xgboostfold2 = load_npz('./5-fold CV/XGboost_folds/RP3Beta/ItItSimilarity_trained_RecommenderFold2_noemptyprofiles_RP3Beta_kaggleMAP0.09372.npz')

S_rp3beta_xgboostfold3 = load_npz('./5-fold CV/XGboost_folds/RP3Beta/ItItSimilarity_trained_RecommenderFold3_noemptyprofiles_RP3Beta_kaggleMAP0.09372.npz')

S_rp3beta_xgboostfold4 = load_npz('./5-fold CV/XGboost_folds/RP3Beta/ItItSimilarity_trained_RecommenderFold4_noemptyprofiles_RP3Beta_kaggleMAP0.09372.npz')

In [29]:
# EASER

S_easer_xgboostfold0 = load_npz('./5-fold CV/XGboost_folds/EASER/ItItSimilarity_trained_RecommenderFold0_noemptyprofiles_EASER_kaggleMAP0.12504.npz')

S_easer_xgboostfold1 = load_npz('./5-fold CV/XGboost_folds/EASER/ItItSimilarity_trained_RecommenderFold1_noemptyprofiles_EASER_kaggleMAP0.12504.npz')

S_easer_xgboostfold2 = load_npz('./5-fold CV/XGboost_folds/EASER/ItItSimilarity_trained_RecommenderFold2_noemptyprofiles_EASER_kaggleMAP0.12504.npz')

S_easer_xgboostfold3 = load_npz('./5-fold CV/XGboost_folds/EASER/ItItSimilarity_trained_RecommenderFold3_noemptyprofiles_EASER_kaggleMAP0.12504.npz')

S_easer_xgboostfold4 = load_npz('./5-fold CV/XGboost_folds/EASER/ItItSimilarity_trained_RecommenderFold4_noemptyprofiles_EASER_kaggleMAP0.12504.npz')

In [30]:
S_models_xgboostfold0 = {
    "SLIM":S_slim_xgboostfold0, "IBCF":S_ibcf_xgboostfold0,"rp3beta":S_rp3beta_xgboostfold0,"EASER":S_easer_xgboostfold0
}

S_models_xgboostfold1 = {
    "SLIM":S_slim_xgboostfold1, "IBCF":S_ibcf_xgboostfold1,"rp3beta":S_rp3beta_xgboostfold1,"EASER":S_easer_xgboostfold1
}

S_models_xgboostfold2 = {
    "SLIM":S_slim_xgboostfold2, "IBCF":S_ibcf_xgboostfold2,"rp3beta":S_rp3beta_xgboostfold2,"EASER":S_easer_xgboostfold2
}

S_models_xgboostfold3 = {
    "SLIM":S_slim_xgboostfold3, "IBCF":S_ibcf_xgboostfold3,"rp3beta":S_rp3beta_xgboostfold3,"EASER":S_easer_xgboostfold3
}

S_models_xgboostfold4 = {
    "SLIM":S_slim_xgboostfold4, "IBCF":S_ibcf_xgboostfold4,"rp3beta":S_rp3beta_xgboostfold4,"EASER":S_easer_xgboostfold4
}

xgboostfold_models = [S_models_xgboostfold0,S_models_xgboostfold1,S_models_xgboostfold2,S_models_xgboostfold3,S_models_xgboostfold4]

In [31]:
import gc

In [32]:
from Recommenders.KNN.ItemKNNCustomSimilarityRecommender import ItemKNNCustomSimilarityRecommender

In [33]:
def precision(recommended_items, relevant_items):

    is_relevant = np.in1d(recommended_items, relevant_items, assume_unique=True)

    precision_score = np.sum(is_relevant, dtype=np.float32) / len(is_relevant)

    return precision_score

def recall(recommended_items, relevant_items):

    is_relevant = np.in1d(recommended_items, relevant_items, assume_unique=True)

    recall_score = np.sum(is_relevant, dtype=np.float32) / relevant_items.shape[0]

    return recall_score

def AP(recommended_items, relevant_items):

    is_relevant = np.in1d(recommended_items, relevant_items, assume_unique=True)

    # Cumulative sum: precision at 1, at 2, at 3 ...
    p_at_k = is_relevant * np.cumsum(is_relevant, dtype=np.float32) / (1 + np.arange(is_relevant.shape[0]))

    ap_score = np.sum(p_at_k) / np.min([relevant_items.shape[0], is_relevant.shape[0]])

    return ap_score

def evaluate_algorithm(URM_test, recommender_object, at=10):

    cumulative_precision = 0.0
    cumulative_recall = 0.0
    cumulative_AP = 0.0

    num_eval = 0


    for user_id in range(URM_test.shape[0]):

        relevant_items = URM_test.indices[URM_test.indptr[user_id]:URM_test.indptr[user_id+1]]

        if len(relevant_items)>0:

            recommended_items = recommender_object.recommend(user_id, at=at,exclude_seen=True)
            num_eval+=1

            cumulative_precision += precision(recommended_items, relevant_items)
            cumulative_recall += recall(recommended_items, relevant_items)
            cumulative_AP += AP(recommended_items, relevant_items)

    mean_precision = cumulative_precision / num_eval
    mean_recall = cumulative_recall / num_eval
    MAP = cumulative_AP / num_eval

    print("Recommender results are: Precision = {:.10f}, Recall = {:.10f}, MAP = {:.10f}".format(
        cumulative_precision, cumulative_recall, MAP))
    return MAP, mean_precision, mean_recall

In [34]:
class Predictor(object):

    def __init__(self, URM, model):
        self.URM = URM
        self.model = model

    def recommend(self, user_id, at=10, exclude_seen=True):
        # Check if user_id is a valid index
        if user_id < 0 or user_id >= self.URM.shape[0]:
            print(f"Invalid user_id: {user_id}")
            return

        # compute the scores using the fitted model
        scores = self.model._compute_item_score(user_id)[0]

        if exclude_seen:
            scores = self.filter_seen(user_id, scores)

        # rank items
        ranking = scores.argsort()[::-1]

        return ranking[:at]


    def filter_seen(self, user_id, scores):

        start_pos = self.URM.indptr[user_id]
        end_pos = self.URM.indptr[user_id+1]

        user_profile = self.URM.indices[start_pos:end_pos]

        scores[user_profile] = -np.inf

        return scores

## Train XGboost model

**Define our best model i.e. NON-NORMALISED SIMILARITIES hybrid 0.685907*(0.88*SLIM elasticnet + 0.12*IBCF) + (1 - 0.685907)EASER**

In [35]:
# Define our best submission model i.e. hybrid of NON-NORMALISED SIMILARITIES 0.685907*(0.88*SLIM elasticnet + 0.12*IBCF)+(1 - 0.685907)*EASER

alpha = 0.88
beta = 0.685907

data_fold0 = data_folds[0]

S_slim_elastic_f0 = S_models_fold0["SLIM"]
S_ibcf_f0 = S_models_fold0["IBCF"]
S_easer_f0 = S_models_fold0["EASER"]
hybrid_similarity = beta * ((1 - alpha) * S_ibcf_f0  + alpha * S_slim_elastic_f0) + (1 - beta) * S_easer_f0

hybrid_recommender_object = ItemKNNCustomSimilarityRecommender(data_fold0['train'])
hybrid_recommender_object.fit(hybrid_similarity)
hybrid_3models = Predictor(URM=data_fold0['train'], model = hybrid_recommender_object)



ItemKNNCustomSimilarityRecommender: URM Detected 205 ( 1.6%) users with no interactions.
ItemKNNCustomSimilarityRecommender: URM Detected 120 ( 0.5%) items with no interactions.


**Build the dataframe with the predictions**

In [ ]:
#!pip install xgboost

In [36]:
import pandas as pd
from tqdm import tqdm
import scipy.sparse as sps
import numpy as np
from xgboost import XGBRanker

n_users, n_items = data_fold0['train'].shape

training_dataframe = pd.DataFrame(index=range(0,n_users), columns = ["ItemID"])
training_dataframe.index.name='UserID'

In [37]:
training_dataframe

,ItemID
UserID,
0,NaN
1,NaN
2,NaN
3,NaN
4,NaN
...,...
12633,NaN
12634,NaN
12635,NaN


In [38]:
cutoff = 30

for user_id in tqdm(range(n_users)):
    recommendations = hybrid_3models.recommend(user_id, at = cutoff)
    training_dataframe.loc[user_id, "ItemID"] = recommendations



100%|██████████| 12638/12638 [00:24<00:00, 513.35it/s]


In [39]:
training_dataframe

,ItemID
UserID,
0,"[587, 674, 1449, 4, 1167, 2755, 1089, 3186, 24..."
1,"[2548, 89, 227, 84, 1831, 812, 887, 83, 3119, ..."
2,"[7347, 5375, 8126, 8303, 189, 4809, 3612, 1073..."
3,"[589, 49, 83, 227, 812, 841, 815, 1425, 5279, ..."
4,"[323, 950, 559, 4316, 1239, 4709, 954, 955, 95..."
...,...
12633,"[2809, 2818, 2807, 2815, 2817, 2811, 4, 22, 30..."
12634,"[2810, 3958, 3943, 3328, 3950, 3955, 3949, 114..."
12635,"[257, 248, 474, 259, 472, 470, 4864, 15770, 83..."


**Let's transform the dataframe so that we have one recommendation per row**

In [40]:
training_dataframe = training_dataframe.explode("ItemID")
training_dataframe['ItemID'] = pd.to_numeric(training_dataframe['ItemID'])
training_dataframe

,ItemID
UserID,
0,587
0,674
0,1449
0,4
0,1167
...,...
12637,1487
12637,821
12637,371


**We should add the target column, which is the correct recommendation as contained in the validation data**

In [41]:
URM_validation_coo = sps.coo_matrix(data_fold0['val'])

correct_recommendations = pd.DataFrame({"UserID": URM_validation_coo.row,
                                        "ItemID": URM_validation_coo.col})
correct_recommendations

,UserID,ItemID
0,0,3
1,0,4
2,0,7
3,0,16
4,0,17
...,...,...
95741,12637,11426
95742,12637,13248
95743,12637,16585
95744,12637,16594


In [42]:
training_dataframe = pd.merge(training_dataframe, correct_recommendations, on=['UserID','ItemID'], how='left', indicator='Exist')
training_dataframe

,UserID,ItemID,Exist
0,0,587,left_only
1,0,674,left_only
2,0,1449,left_only
3,0,4,both
4,0,1167,left_only
...,...,...,...
379135,12637,1487,both
379136,12637,821,left_only
379137,12637,371,left_only
379138,12637,13255,left_only


In [43]:
training_dataframe["Label"] = training_dataframe["Exist"] == "both"
training_dataframe.drop(columns = ['Exist'], inplace=True)
training_dataframe

,UserID,ItemID,Label
0,0,587,False
1,0,674,False
2,0,1449,False
3,0,4,True
4,0,1167,False
...,...,...,...
379135,12637,1487,True
379136,12637,821,False
379137,12637,371,False
379138,12637,13255,False


**Now let's add features as the prediction of other algorithms**

In [44]:
S_rp3beta_elastic_f0 = S_models_fold0["rp3beta"]
rp3beta_recommender_object = ItemKNNCustomSimilarityRecommender(data_fold0['train'])
rp3beta_recommender_object.fit(S_rp3beta_elastic_f0)

S_ibcf_f0 = S_models_fold0["IBCF"]
ibcf_recommender_object = ItemKNNCustomSimilarityRecommender(data_fold0['train'])
ibcf_recommender_object.fit(S_ibcf_f0)

S_easer_f0 = S_models_fold0["EASER"]
easer_recommender_object = ItemKNNCustomSimilarityRecommender(data_fold0['train'])
easer_recommender_object.fit(S_easer_f0)



other_algorithms = {
    "rp3beta": rp3beta_recommender_object,
    "EASER": easer_recommender_object,
    "IBCF":ibcf_recommender_object
}


ItemKNNCustomSimilarityRecommender: URM Detected 205 ( 1.6%) users with no interactions.
ItemKNNCustomSimilarityRecommender: URM Detected 120 ( 0.5%) items with no interactions.
ItemKNNCustomSimilarityRecommender: URM Detected 205 ( 1.6%) users with no interactions.
ItemKNNCustomSimilarityRecommender: URM Detected 120 ( 0.5%) items with no interactions.
ItemKNNCustomSimilarityRecommender: URM Detected 205 ( 1.6%) users with no interactions.
ItemKNNCustomSimilarityRecommender: URM Detected 120 ( 0.5%) items with no interactions.


In [45]:
training_dataframe = training_dataframe.set_index('UserID')

for user_id in tqdm(range(n_users)):
    for rec_label, rec_instance in other_algorithms.items():

        item_list = training_dataframe.loc[user_id, "ItemID"].values.tolist()

        all_item_scores = rec_instance._compute_item_score([user_id], items_to_compute = item_list)

        training_dataframe.loc[user_id, rec_label] = all_item_scores[0, item_list]

training_dataframe = training_dataframe.reset_index()
training_dataframe = training_dataframe.rename(columns = {"index": "UserID"})
training_dataframe

  0%|          | 0/12638 [00:00<?, ?it/s]

100%|██████████| 12638/12638 [00:41<00:00, 302.28it/s]


,UserID,ItemID,Label,rp3beta,EASER,IBCF
0,0,587,False,1.756946,0.384206,0.885922
1,0,674,False,1.890487,0.325518,0.612876
2,0,1449,False,1.065335,0.203652,0.650260
3,0,4,True,1.535873,0.230577,0.489168
4,0,1167,False,1.207187,0.243150,0.380579
...,...,...,...,...,...,...
379135,12637,1487,True,1.885228,0.125901,0.877917
379136,12637,821,False,1.422612,0.190398,0.547981
379137,12637,371,False,1.501966,0.162347,0.767767
379138,12637,13255,False,1.306112,0.090506,1.258470


**The popularity of the item, the user activity and etc. features**

In [46]:
item_popularity = np.ediff1d(sps.csc_matrix(data_fold0['train']).indptr)

training_dataframe['item_popularity'] = item_popularity[training_dataframe["ItemID"].values.astype(int)]
training_dataframe

,UserID,ItemID,Label,rp3beta,EASER,IBCF,item_popularity
0,0,587,False,1.756946,0.384206,0.885922,586
1,0,674,False,1.890487,0.325518,0.612876,342
2,0,1449,False,1.065335,0.203652,0.650260,97
3,0,4,True,1.535873,0.230577,0.489168,208
4,0,1167,False,1.207187,0.243150,0.380579,274
...,...,...,...,...,...,...,...
379135,12637,1487,True,1.885228,0.125901,0.877917,64
379136,12637,821,False,1.422612,0.190398,0.547981,147
379137,12637,371,False,1.501966,0.162347,0.767767,36
379138,12637,13255,False,1.306112,0.090506,1.258470,6


**The profile length**

In [47]:
user_popularity = np.ediff1d(sps.csr_matrix(data_fold0['train']).indptr)

training_dataframe['user_profile_len'] = user_popularity[training_dataframe["UserID"].values.astype(int)]
training_dataframe

,UserID,ItemID,Label,rp3beta,EASER,IBCF,item_popularity,user_profile_len
0,0,587,False,1.756946,0.384206,0.885922,586,36
1,0,674,False,1.890487,0.325518,0.612876,342,36
2,0,1449,False,1.065335,0.203652,0.650260,97,36
3,0,4,True,1.535873,0.230577,0.489168,208,36
4,0,1167,False,1.207187,0.243150,0.380579,274,36
...,...,...,...,...,...,...,...,...
379135,12637,1487,True,1.885228,0.125901,0.877917,64,52
379136,12637,821,False,1.422612,0.190398,0.547981,147,52
379137,12637,371,False,1.501966,0.162347,0.767767,36,52
379138,12637,13255,False,1.306112,0.090506,1.258470,6,52


**Then we train XGBoost to rerank those prediction using as lable whether they should be recommended or not**

In [48]:
training_dataframe = training_dataframe.sort_values("UserID").reset_index()
training_dataframe.drop(columns = ['index'], inplace=True)
training_dataframe

,UserID,ItemID,Label,rp3beta,EASER,IBCF,item_popularity,user_profile_len
0,0,587,False,1.756946,0.384206,0.885922,586,36
1,0,2441,False,0.224008,0.081729,0.139770,148,36
2,0,353,False,0.323550,0.097789,0.307937,71,36
3,0,818,False,0.355252,0.075370,0.206294,227,36
4,0,4512,False,0.726684,0.076388,0.393346,56,36
...,...,...,...,...,...,...,...,...
379135,12637,9567,False,2.224067,0.254522,1.413900,37,52
379136,12637,10897,True,2.262047,0.280913,1.609995,37,52
379137,12637,13255,False,1.306112,0.090506,1.258470,6,52
379138,12637,17581,True,2.187028,0.160564,1.387581,16,52


**To use the ranker one first needs to specify the size of the groups, a group is the dimension you rank on, in this case each group corresponds to a user. Since we have generated a fixed number of candidate items for each user all groups have the same length.**

In [49]:
groups = training_dataframe.groupby("UserID").size().values
groups

array([30, 30, 30, ..., 30, 30, 30], dtype=int64)

In [50]:
from xgboost import XGBRanker

In [51]:
n_estimators = 50
learning_rate = 1e-1
reg_alpha = 1e-1
reg_lambda = 1e-1
max_depth = 5
max_leaves = 0
grow_policy = "depthwise"
objective = "pairwise"
booster = "gbtree"
use_user_profile = False
random_seed = None

XGB_model = XGBRanker(objective='rank:{}'.format(objective),
                      n_estimators = int(n_estimators),
                      random_state = random_seed,
                      learning_rate = learning_rate,
                      reg_alpha = reg_alpha,
                      reg_lambda = reg_lambda,
                      max_depth = int(max_depth),
                      max_leaves = int(max_leaves),
                      grow_policy = grow_policy,
                      verbosity = 0, # 2 if self.verbose else 0,
                      booster = booster,
                      )

In [52]:
y_train = training_dataframe["Label"]
X_train = training_dataframe.drop(columns=["Label"])

XGB_model.fit(X_train,
          y_train,
          group=groups,
          verbose=True)

XGBRanker(base_score=None, booster='gbtree', callbacks=None,
          colsample_bylevel=None, colsample_bynode=None, colsample_bytree=None,
          device=None, early_stopping_rounds=None, enable_categorical=False,
          eval_metric=None, feature_types=None, gamma=None,
          grow_policy='depthwise', importance_type=None,
          interaction_constraints=None, learning_rate=0.1, max_bin=None,
          max_cat_threshold=None, max_cat_to_onehot=None, max_delta_step=None,
          max_depth=5, max_leaves=0, min_child_weight=None, missing=nan,
          monotone_constraints=None, multi_strategy=None, n_estimators=50,
          n_jobs=None, num_parallel_tree=None, objective='rank:pairwise', ...)

In [53]:
X_train["UserID"] = X_train["UserID"].astype("category")
X_train["ItemID"] = X_train["ItemID"].astype("category")

In [54]:
XGB_model = XGBRanker(objective='rank:{}'.format(objective),
                      n_estimators = int(n_estimators),
                      random_state = random_seed,
                      learning_rate = learning_rate,
                      reg_alpha = reg_alpha,
                      reg_lambda = reg_lambda,
                      max_depth = int(max_depth),
                      max_leaves = int(max_leaves),
                      grow_policy = grow_policy,
                      verbosity = 0, # 2 if self.verbose else 0,
                      booster = booster,
                      enable_categorical = True,
                      tree_method = "hist",  # Supported tree methods are `gpu_hist`, `approx`, and `hist`.
                      )

XGB_model.fit(X_train,
          y_train,
          group=groups,
          verbose=True)

XGBRanker(base_score=None, booster='gbtree', callbacks=None,
          colsample_bylevel=None, colsample_bynode=None, colsample_bytree=None,
          device=None, early_stopping_rounds=None, enable_categorical=True,
          eval_metric=None, feature_types=None, gamma=None,
          grow_policy='depthwise', importance_type=None,
          interaction_constraints=None, learning_rate=0.1, max_bin=None,
          max_cat_threshold=None, max_cat_to_onehot=None, max_delta_step=None,
          max_depth=5, max_leaves=0, min_child_weight=None, missing=nan,
          monotone_constraints=None, multi_strategy=None, n_estimators=50,
          n_jobs=None, num_parallel_tree=None, objective='rank:pairwise', ...)

## Evaluate the XGboost model

In [55]:
def evaluate_xgboost(URM_test,feature_dataframe, xgboost_model, at=10):

    cumulative_precision = 0.0
    cumulative_recall = 0.0
    cumulative_AP = 0.0

    num_eval = 0

    for user_id in range(URM_test.shape[0]):

        relevant_items = URM_test.indices[URM_test.indptr[user_id]:URM_test.indptr[user_id+1]]

        if len(relevant_items)>0:

          # Extract the predictions and features for the user
          X_to_predict = feature_dataframe[feature_dataframe["UserID"] == user_id]
          # Do the reranking prediction
          item_reranking_scores = xgboost_model.predict(X_to_predict)
          if len(item_reranking_scores) == 0:
            print(f'Lenght of reranking for user {usr_idx} is 0')
          # Rerank the items according to the reranking scores
          initial_recommendations = X_to_predict.ItemID.to_list()
          reranked_recommendations = np.array(initial_recommendations)[np.argsort(item_reranking_scores)[::-1]]

          num_eval+=1

          cumulative_precision += precision(reranked_recommendations, relevant_items)
          cumulative_recall += recall(reranked_recommendations, relevant_items)
          cumulative_AP += AP(reranked_recommendations, relevant_items)

    mean_precision = cumulative_precision / num_eval
    mean_recall = cumulative_recall / num_eval
    MAP = cumulative_AP / num_eval

    print("Recommender results are: Precision = {:.10f}, Recall = {:.10f}, MAP = {:.10f}".format(
        cumulative_precision, cumulative_recall, MAP))

    return MAP, mean_precision, mean_recall

In [56]:
# Define the function for training XGboost model
from xgboost import XGBRanker
from tqdm import tqdm

def prepare_training_df(urm_train_recommender,urm_train_xgboost,initial_recommender,initial_recommendations_cutoff,other_algorithms):
  # Build the dataframe with the predictions
  n_users, n_items = urm_train_recommender.shape
  training_dataframe = pd.DataFrame(index=range(0,n_users), columns = ["ItemID"])
  training_dataframe.index.name='UserID'

  for user_id in tqdm(range(n_users)):
    recommendations = initial_recommender.recommend(user_id, at = initial_recommendations_cutoff)
    training_dataframe.loc[user_id, "ItemID"] = recommendations

  # Let's transform the dataframe so that we have one recommendation per row
  training_dataframe = training_dataframe.explode("ItemID")
  training_dataframe['ItemID'] = pd.to_numeric(training_dataframe['ItemID'])

  # We should add the target column, which is the correct recommendation as contained in the validation data
  URM_train_xgboost_coo = sps.coo_matrix(urm_train_xgboost)
  correct_recommendations = pd.DataFrame({"UserID": URM_train_xgboost_coo.row,
                                        "ItemID": URM_train_xgboost_coo.col})

  training_dataframe = pd.merge(training_dataframe, correct_recommendations, on=['UserID','ItemID'], how='left', indicator='Exist')
  training_dataframe["Label"] = training_dataframe["Exist"] == "both"
  training_dataframe.drop(columns = ['Exist'], inplace=True)

  # Now let's add features as the prediction of other algorithms

  training_dataframe = training_dataframe.set_index('UserID')

  for user_id in tqdm(range(n_users)):
    for rec_label, rec_instance in other_algorithms.items():
      item_list = training_dataframe.loc[user_id, "ItemID"].values.tolist()

      all_item_scores = rec_instance._compute_item_score([user_id], items_to_compute = item_list)

      training_dataframe.loc[user_id, rec_label] = all_item_scores[0, item_list]

  training_dataframe = training_dataframe.reset_index()
  training_dataframe = training_dataframe.rename(columns = {"index": "UserID"})

  # The popularity of the item, the user activity and other features
  item_popularity = np.ediff1d(sps.csc_matrix(urm_train_recommender).indptr)
  training_dataframe['item_popularity'] = item_popularity[training_dataframe["ItemID"].values.astype(int)]

  # The profile length
  user_popularity = np.ediff1d(sps.csr_matrix(urm_train_recommender).indptr)
  training_dataframe['user_profile_len'] = user_popularity[training_dataframe["UserID"].values.astype(int)]

  # Then we train XGBoost to rerank those prediction using as label whether they should be recommended or not
  training_dataframe = training_dataframe.sort_values("UserID").reset_index()
  training_dataframe.drop(columns = ['index'], inplace=True)

  return training_dataframe


def train_xgboost(urm_train_recommender, urm_train_xgboost,initial_recommender, initial_recommendations_cutoff, other_algorithms,
                  xgboost_hyperparams):

  training_dataframe = prepare_training_df(urm_train_recommender, urm_train_xgboost,initial_recommender, initial_recommendations_cutoff, other_algorithms)
  y_train = training_dataframe["Label"]
  X_train = training_dataframe.drop(columns=["Label"])

  # To use the ranker one first needs to specify the size of the groups, a group is the dimension you rank on.
  # In this case each group corresponds to a user. Since we have generated a fixed number of candidate items for each user all groups have the same length.
  groups = training_dataframe.groupby("UserID").size().values


  use_user_profile = xgboost_hyperparams['use_user_profile']
  random_seed = xgboost_hyperparams['random_seed']

  XGB_model = XGBRanker(objective='rank:{}'.format(xgboost_hyperparams['objective']),
                      n_estimators = int(xgboost_hyperparams['n_estimators']),
                      random_state = random_seed,
                      learning_rate = xgboost_hyperparams['learning_rate'],
                      reg_alpha = xgboost_hyperparams['reg_alpha'],
                      reg_lambda = xgboost_hyperparams['reg_lambda'],
                      max_depth = int(xgboost_hyperparams['max_depth']),
                      max_leaves = int(xgboost_hyperparams['max_leaves']),
                      grow_policy = xgboost_hyperparams['grow_policy'],
                      verbosity = 0, # 2 if self.verbose else 0,
                      booster = xgboost_hyperparams['booster'],
                      )


  XGB_model.fit(X_train,
          y_train,
          group=groups,
          verbose=True)

  return XGB_model

In [57]:
# Evaluate model recommendations with 5-fold CV
# NON-Normalised SIMILARITIES
# USING SPECIFIED VALUES (MAX ONE'S)
alpha = 0.88
beta = 0.685907

init_rec_cutoff = 100

xgboost_model_hyperparams = {'n_estimators':50,
                             'learning_rate':1e-1,'reg_alpha':1e-1,'reg_lambda':1e-1,'max_depth':5,'max_leaves':0,
                             'grow_policy':"depthwise",'objective':"pairwise",'booster':"gbtree",
                             'use_user_profile':False,'random_seed':None
                             }

folds_map = []
for k,fold in enumerate(data_folds):
  print(f'Fold {k} evaluations started')

  # Define the recommender models trained on RecommenderFold of the current fold to prepare for XGboost training
  initial_S_slim_elastic = xgboostfold_models[k]["SLIM"]
  initial_S_ibcf = xgboostfold_models[k]["IBCF"]
  initial_S_easer = xgboostfold_models[k]["EASER"]
  initial_hybrid3models_similarity = beta * ((1 - alpha) * initial_S_ibcf  + alpha * initial_S_slim_elastic) + (1 - beta) * initial_S_easer


  # Models for features generation
  initial_hybrid3models_object = ItemKNNCustomSimilarityRecommender(data_xgboostfolds[k]['train_recommender'])
  initial_hybrid3models_object.fit(initial_hybrid3models_similarity)
  initial_hybrid3models = Predictor(URM=data_xgboostfolds[k]['train_recommender'], model = initial_hybrid3models_object)

  initial_S_rp3beta = xgboostfold_models[k]["rp3beta"]
  initial_rp3beta_object = ItemKNNCustomSimilarityRecommender(data_xgboostfolds[k]['train_recommender'])
  initial_rp3beta_object.fit(initial_S_rp3beta)

  initial_S_easer = xgboostfold_models[k]["EASER"]
  initial_easer_object = ItemKNNCustomSimilarityRecommender(data_xgboostfolds[k]['train_recommender'])
  initial_easer_object.fit(initial_S_easer)

  initial_S_ibcf = xgboostfold_models[k]["IBCF"]
  initial_ibcf_object = ItemKNNCustomSimilarityRecommender(data_xgboostfolds[k]['train_recommender'])
  initial_ibcf_object.fit(initial_S_ibcf)

  init_other_algorithms = {
    "rp3beta": initial_rp3beta_object,
    "EASER": initial_easer_object,
    "IBCF":initial_ibcf_object}

  # Get XGboost trained
  xgboost_trained = train_xgboost(data_xgboostfolds[k]['train_recommender'],data_xgboostfolds[k]['train_xgboost'],
                                  initial_hybrid3models,initial_recommendations_cutoff = init_rec_cutoff, other_algorithms = init_other_algorithms,
                                  xgboost_hyperparams = xgboost_model_hyperparams)


  # Define the recommender models trained on the whole current fold to prepare for the evaluation
  S_slim_elastic = fold_models[k]["SLIM"]
  S_ibcf = fold_models[k]["IBCF"]
  S_easer = fold_models[k]["EASER"]
  hybrid3models_similarity = beta * ((1 - alpha) * S_ibcf  + alpha * S_slim_elastic) + (1 - beta) * S_easer

  hybrid3models_object = ItemKNNCustomSimilarityRecommender(fold['train'])
  hybrid3models_object.fit(hybrid3models_similarity)
  hybrid3models = Predictor(URM=fold['train'], model = hybrid3models_object)

  # Models for features generation
  S_rp3beta = fold_models[k]["rp3beta"]
  rp3beta_object = ItemKNNCustomSimilarityRecommender(fold['train'])
  rp3beta_object.fit(S_rp3beta)

  S_easer = fold_models[k]["EASER"]
  easer_object = ItemKNNCustomSimilarityRecommender(fold['train'])
  easer_object.fit(S_easer)

  S_ibcf = fold_models[k]["IBCF"]
  ibcf_object = ItemKNNCustomSimilarityRecommender(fold['train'])
  ibcf_object.fit(S_ibcf)

  other_algorithms = {
    "rp3beta": rp3beta_object,
    "EASER": easer_object,
    "IBCF":ibcf_object}

  features_df = prepare_training_df(fold['train'],fold['train'],hybrid3models,init_rec_cutoff,other_algorithms)
  features_df = features_df.drop(columns=["Label"])


  map, mp,mr = evaluate_xgboost(fold['val'],features_df,xgboost_trained)
  folds_map.append(map)


cv_map = np.array(folds_map).sum()/len(data_folds)
print(f'The crossvalidation MAP for list combination of Noramalised IBCF and SLIM elasticnet is {cv_map}')

Fold 0 evaluations started
ItemKNNCustomSimilarityRecommender: URM Detected 424 ( 3.4%) users with no interactions.
ItemKNNCustomSimilarityRecommender: URM Detected 334 ( 1.5%) items with no interactions.
ItemKNNCustomSimilarityRecommender: URM Detected 424 ( 3.4%) users with no interactions.
ItemKNNCustomSimilarityRecommender: URM Detected 334 ( 1.5%) items with no interactions.
ItemKNNCustomSimilarityRecommender: URM Detected 424 ( 3.4%) users with no interactions.
ItemKNNCustomSimilarityRecommender: URM Detected 334 ( 1.5%) items with no interactions.
ItemKNNCustomSimilarityRecommender: URM Detected 424 ( 3.4%) users with no interactions.
ItemKNNCustomSimilarityRecommender: URM Detected 334 ( 1.5%) items with no interactions.


100%|██████████| 12638/12638 [00:42<00:00, 299.96it/s]


ItemKNNCustomSimilarityRecommender: URM Detected 205 ( 1.6%) users with no interactions.
ItemKNNCustomSimilarityRecommender: URM Detected 120 ( 0.5%) items with no interactions.
ItemKNNCustomSimilarityRecommender: URM Detected 205 ( 1.6%) users with no interactions.
ItemKNNCustomSimilarityRecommender: URM Detected 120 ( 0.5%) items with no interactions.
ItemKNNCustomSimilarityRecommender: URM Detected 205 ( 1.6%) users with no interactions.
ItemKNNCustomSimilarityRecommender: URM Detected 120 ( 0.5%) items with no interactions.
ItemKNNCustomSimilarityRecommender: URM Detected 205 ( 1.6%) users with no interactions.
ItemKNNCustomSimilarityRecommender: URM Detected 120 ( 0.5%) items with no interactions.


100%|██████████| 12638/12638 [00:38<00:00, 327.34it/s]


Recommender results are: Precision = 308.1200000000, Recall = 3372.4768959413, MAP = 0.0876053711
Fold 1 evaluations started
ItemKNNCustomSimilarityRecommender: URM Detected 464 ( 3.7%) users with no interactions.
ItemKNNCustomSimilarityRecommender: URM Detected 345 ( 1.6%) items with no interactions.
ItemKNNCustomSimilarityRecommender: URM Detected 464 ( 3.7%) users with no interactions.
ItemKNNCustomSimilarityRecommender: URM Detected 345 ( 1.6%) items with no interactions.
ItemKNNCustomSimilarityRecommender: URM Detected 464 ( 3.7%) users with no interactions.
ItemKNNCustomSimilarityRecommender: URM Detected 345 ( 1.6%) items with no interactions.
ItemKNNCustomSimilarityRecommender: URM Detected 464 ( 3.7%) users with no interactions.
ItemKNNCustomSimilarityRecommender: URM Detected 345 ( 1.6%) items with no interactions.


...

Recommender results are: Precision = 308.7400000000, Recall = 3423.4580844912, MAP = 0.0875540715
Fold 4 evaluations started
ItemKNNCustomSimilarityRecommender: URM Detected 478 ( 3.8%) users with no interactions.
ItemKNNCustomSimilarityRecommender: URM Detected 337 ( 1.5%) items with no interactions.
ItemKNNCustomSimilarityRecommender: URM Detected 478 ( 3.8%) users with no interactions.
ItemKNNCustomSimilarityRecommender: URM Detected 337 ( 1.5%) items with no interactions.
ItemKNNCustomSimilarityRecommender: URM Detected 478 ( 3.8%) users with no interactions.
ItemKNNCustomSimilarityRecommender: URM Detected 337 ( 1.5%) items with no interactions.
ItemKNNCustomSimilarityRecommender: URM Detected 478 ( 3.8%) users with no interactions.
ItemKNNCustomSimilarityRecommender: URM Detected 337 ( 1.5%) items with no interactions.


100%|██████████| 12638/12638 [00:44<00:00, 281.68it/s]


ItemKNNCustomSimilarityRecommender: URM Detected 239 ( 1.9%) users with no interactions.
ItemKNNCustomSimilarityRecommender: URM Detected 124 ( 0.6%) items with no interactions.
ItemKNNCustomSimilarityRecommender: URM Detected 239 ( 1.9%) users with no interactions.
ItemKNNCustomSimilarityRecommender: URM Detected 124 ( 0.6%) items with no interactions.
ItemKNNCustomSimilarityRecommender: URM Detected 239 ( 1.9%) users with no interactions.
ItemKNNCustomSimilarityRecommender: URM Detected 124 ( 0.6%) items with no interactions.
ItemKNNCustomSimilarityRecommender: URM Detected 239 ( 1.9%) users with no interactions.
ItemKNNCustomSimilarityRecommender: URM Detected 124 ( 0.6%) items with no interactions.


100%|██████████| 12638/12638 [00:43<00:00, 292.18it/s]


Recommender results are: Precision = 306.5000000000, Recall = 3388.8115602459, MAP = 0.0886982569
The crossvalidation MAP for list combination of Noramalised IBCF and SLIM elasticnet is 0.0873284097545886


In [58]:
from scipy.stats import loguniform
from scipy.stats import uniform

In [59]:
rnd_init_rec_cutoff = np.array(list(set(uniform.rvs(100,200, size=3).astype(int))))
rnd_init_rec_cutoff = [200, 600]
print(f"Init_rec_cutoff:{rnd_init_rec_cutoff}")

Init_rec_cutoff:[200, 600]


In [61]:
rnd_init_rec_reg_alpha = [0.1, 0.3, 0.001]
rnd_init_rec_reg_beta = [0.1, 0.3, 0.001]
print(f"Init_rec_reg_alpha:{rnd_init_rec_reg_alpha}")
print(f"Init_rec_reg_beta:{rnd_init_rec_reg_beta}")

Init_rec_reg_alpha:[0.1, 0.3, 0.001]
Init_rec_reg_beta:[0.1, 0.3, 0.001]


In [62]:
results_list = []

In [69]:
for init_rec_cutoff in rnd_init_rec_cutoff:
  for init_rec_reg_alpha in rnd_init_rec_reg_alpha:
    for init_rec_reg_beta in rnd_init_rec_reg_beta:
      xgboost_model_hyperparams = {'n_estimators':50,
                             'learning_rate':1e-1,'reg_alpha':init_rec_reg_alpha,'reg_lambda':init_rec_reg_beta,'max_depth':5,'max_leaves':0,
                             'grow_policy':"depthwise",'objective':"pairwise",'booster':"gbtree",
                             'use_user_profile':False,'random_seed':None
                             }
      folds_map = []

      for k,fold in enumerate(data_folds):
        print(f'Fold {k} evaluations started')

        # Define the recommender models trained on RecommenderFold of the current fold to prepare for XGboost training
        initial_S_slim_elastic = xgboostfold_models[k]["SLIM"]
        initial_S_ibcf = xgboostfold_models[k]["IBCF"]
        initial_S_easer = xgboostfold_models[k]["EASER"]
        initial_hybrid3models_similarity = beta * ((1 - alpha) * initial_S_ibcf  + alpha * initial_S_slim_elastic) + (1 - beta) * initial_S_easer


        # Models for features generation
        initial_hybrid3models_object = ItemKNNCustomSimilarityRecommender(data_xgboostfolds[k]['train_recommender'])
        initial_hybrid3models_object.fit(initial_hybrid3models_similarity)
        initial_hybrid3models = Predictor(URM=data_xgboostfolds[k]['train_recommender'], model = initial_hybrid3models_object)

        initial_S_rp3beta = xgboostfold_models[k]["rp3beta"]
        initial_rp3beta_object = ItemKNNCustomSimilarityRecommender(data_xgboostfolds[k]['train_recommender'])
        initial_rp3beta_object.fit(initial_S_rp3beta)

        initial_S_easer = xgboostfold_models[k]["EASER"]
        initial_easer_object = ItemKNNCustomSimilarityRecommender(data_xgboostfolds[k]['train_recommender'])
        initial_easer_object.fit(initial_S_easer)

        initial_S_ibcf = xgboostfold_models[k]["IBCF"]
        initial_ibcf_object = ItemKNNCustomSimilarityRecommender(data_xgboostfolds[k]['train_recommender'])
        initial_ibcf_object.fit(initial_S_ibcf)

        init_other_algorithms = {
        "rp3beta": initial_rp3beta_object,
        "EASER": initial_easer_object,
        "IBCF":initial_ibcf_object}

        # Get XGboost trained
        xgboost_trained = train_xgboost(data_xgboostfolds[k]['train_recommender'],data_xgboostfolds[k]['train_xgboost'],
                                      initial_hybrid3models,initial_recommendations_cutoff = init_rec_cutoff, other_algorithms = init_other_algorithms,
                                      xgboost_hyperparams = xgboost_model_hyperparams)


        # Define the recommender models trained on the whole current fold to prepare for the evaluation
        S_slim_elastic = fold_models[k]["SLIM"]
        S_ibcf = fold_models[k]["IBCF"]
        S_easer = fold_models[k]["EASER"]
        hybrid3models_similarity = beta * ((1 - alpha) * S_ibcf  + alpha * S_slim_elastic) + (1 - beta) * S_easer

        hybrid3models_object = ItemKNNCustomSimilarityRecommender(fold['train'])
        hybrid3models_object.fit(hybrid3models_similarity)
        hybrid3models = Predictor(URM=fold['train'], model = hybrid3models_object)

        # Models for features generation
        S_rp3beta = fold_models[k]["rp3beta"]
        rp3beta_object = ItemKNNCustomSimilarityRecommender(fold['train'])
        rp3beta_object.fit(S_rp3beta)

        S_easer = fold_models[k]["EASER"]
        easer_object = ItemKNNCustomSimilarityRecommender(fold['train'])
        easer_object.fit(S_easer)

        S_ibcf = fold_models[k]["IBCF"]
        ibcf_object = ItemKNNCustomSimilarityRecommender(fold['train'])
        ibcf_object.fit(S_ibcf)

        other_algorithms = {
        "rp3beta": rp3beta_object,
        "EASER": easer_object,
        "IBCF":ibcf_object}

        features_df = prepare_training_df(fold['train'],fold['train'],hybrid3models,init_rec_cutoff,other_algorithms)
        features_df = features_df.drop(columns=["Label"])


        map, mp,mr = evaluate_xgboost(fold['val'],features_df,xgboost_trained)
        folds_map.append(map)

        break

      cv_map = np.array(folds_map).sum()/len(data_folds)
      print(f'The crossvalidation MAP for {init_rec_cutoff} is {cv_map}')
      # Append the results to the list
      results_list.append({"cutoff": init_rec_cutoff, "init_rec_reg_alpha": init_rec_reg_alpha, "init_rec_reg_beta": init_rec_reg_beta, "MAP": cv_map})


Fold 0 evaluations started
ItemKNNCustomSimilarityRecommender: URM Detected 424 ( 3.4%) users with no interactions.
ItemKNNCustomSimilarityRecommender: URM Detected 334 ( 1.5%) items with no interactions.
ItemKNNCustomSimilarityRecommender: URM Detected 424 ( 3.4%) users with no interactions.
ItemKNNCustomSimilarityRecommender: URM Detected 334 ( 1.5%) items with no interactions.
ItemKNNCustomSimilarityRecommender: URM Detected 424 ( 3.4%) users with no interactions.
ItemKNNCustomSimilarityRecommender: URM Detected 334 ( 1.5%) items with no interactions.
ItemKNNCustomSimilarityRecommender: URM Detected 424 ( 3.4%) users with no interactions.
ItemKNNCustomSimilarityRecommender: URM Detected 334 ( 1.5%) items with no interactions.


100%|██████████| 12638/12638 [00:41<00:00, 302.15it/s]


ItemKNNCustomSimilarityRecommender: URM Detected 205 ( 1.6%) users with no interactions.
ItemKNNCustomSimilarityRecommender: URM Detected 120 ( 0.5%) items with no interactions.
ItemKNNCustomSimilarityRecommender: URM Detected 205 ( 1.6%) users with no interactions.
ItemKNNCustomSimilarityRecommender: URM Detected 120 ( 0.5%) items with no interactions.
ItemKNNCustomSimilarityRecommender: URM Detected 205 ( 1.6%) users with no interactions.
ItemKNNCustomSimilarityRecommender: URM Detected 120 ( 0.5%) items with no interactions.
ItemKNNCustomSimilarityRecommender: URM Detected 205 ( 1.6%) users with no interactions.
ItemKNNCustomSimilarityRecommender: URM Detected 120 ( 0.5%) items with no interactions.


100%|██████████| 12638/12638 [00:41<00:00, 302.99it/s]


Recommender results are: Precision = 189.6750000000, Recall = 3949.2728899652, MAP = 0.0880983728
The crossvalidation MAP for 200 is 0.017619674551043064
Fold 0 evaluations started
ItemKNNCustomSimilarityRecommender: URM Detected 424 ( 3.4%) users with no interactions.
ItemKNNCustomSimilarityRecommender: URM Detected 334 ( 1.5%) items with no interactions.
ItemKNNCustomSimilarityRecommender: URM Detected 424 ( 3.4%) users with no interactions.
ItemKNNCustomSimilarityRecommender: URM Detected 334 ( 1.5%) items with no interactions.
ItemKNNCustomSimilarityRecommender: URM Detected 424 ( 3.4%) users with no interactions.
ItemKNNCustomSimilarityRecommender: URM Detected 334 ( 1.5%) items with no interactions.
ItemKNNCustomSimilarityRecommender: URM Detected 424 ( 3.4%) users with no interactions.
ItemKNNCustomSimilarityRecommender: URM Detected 334 ( 1.5%) items with no interactions.


...

Recommender results are: Precision = 81.2283333333, Recall = 4817.3488569762, MAP = 0.0900725162
The crossvalidation MAP for 600 is 0.018014503238560305
Fold 0 evaluations started
ItemKNNCustomSimilarityRecommender: URM Detected 424 ( 3.4%) users with no interactions.
ItemKNNCustomSimilarityRecommender: URM Detected 334 ( 1.5%) items with no interactions.
ItemKNNCustomSimilarityRecommender: URM Detected 424 ( 3.4%) users with no interactions.
ItemKNNCustomSimilarityRecommender: URM Detected 334 ( 1.5%) items with no interactions.
ItemKNNCustomSimilarityRecommender: URM Detected 424 ( 3.4%) users with no interactions.
ItemKNNCustomSimilarityRecommender: URM Detected 334 ( 1.5%) items with no interactions.
ItemKNNCustomSimilarityRecommender: URM Detected 424 ( 3.4%) users with no interactions.
ItemKNNCustomSimilarityRecommender: URM Detected 334 ( 1.5%) items with no interactions.


100%|██████████| 12638/12638 [00:47<00:00, 264.88it/s]


ItemKNNCustomSimilarityRecommender: URM Detected 205 ( 1.6%) users with no interactions.
ItemKNNCustomSimilarityRecommender: URM Detected 120 ( 0.5%) items with no interactions.
ItemKNNCustomSimilarityRecommender: URM Detected 205 ( 1.6%) users with no interactions.
ItemKNNCustomSimilarityRecommender: URM Detected 120 ( 0.5%) items with no interactions.
ItemKNNCustomSimilarityRecommender: URM Detected 205 ( 1.6%) users with no interactions.
ItemKNNCustomSimilarityRecommender: URM Detected 120 ( 0.5%) items with no interactions.
ItemKNNCustomSimilarityRecommender: URM Detected 205 ( 1.6%) users with no interactions.
ItemKNNCustomSimilarityRecommender: URM Detected 120 ( 0.5%) items with no interactions.


100%|██████████| 12638/12638 [00:47<00:00, 264.96it/s]


Recommender results are: Precision = 81.2283333333, Recall = 4817.3488569762, MAP = 0.0904468444
The crossvalidation MAP for 600 is 0.018089368884681947


In [71]:
for result in results_list:
  result['MAP'] *= 5

In [72]:
results_list

[{'cutoff': 200,
  'init_rec_reg_alpha': 0.1,
  'init_rec_reg_beta': 0.1,
  'MAP': 0.08809837275521532},
 {'cutoff': 200,
  'init_rec_reg_alpha': 0.1,
  'init_rec_reg_beta': 0.3,
  'MAP': 0.08915430906322261},
 {'cutoff': 200,
  'init_rec_reg_alpha': 0.1,
  'init_rec_reg_beta': 0.001,
  'MAP': 0.08874540179974084},
 {'cutoff': 200,
  'init_rec_reg_alpha': 0.3,
  'init_rec_reg_beta': 0.1,
  'MAP': 0.08837997115491691},
 {'cutoff': 200,
  'init_rec_reg_alpha': 0.3,
  'init_rec_reg_beta': 0.3,
  'MAP': 0.0890542247338913},
 {'cutoff': 200,
  'init_rec_reg_alpha': 0.3,
  'init_rec_reg_beta': 0.001,
  'MAP': 0.08899401821310435},
 {'cutoff': 200,
  'init_rec_reg_alpha': 0.001,
  'init_rec_reg_beta': 0.1,
  'MAP': 0.08871391302467713},
 {'cutoff': 200,
  'init_rec_reg_alpha': 0.001,
  'init_rec_reg_beta': 0.3,
  'MAP': 0.08817960297534101},
 {'cutoff': 200,
  'init_rec_reg_alpha': 0.001,
  'init_rec_reg_beta': 0.001,
  'MAP': 0.0888874217281479},
 {'cutoff': 600,
  'init_rec_reg_alpha': 0.1,

In [73]:
# Find the dictionary with the highest 'MAP' value
max_result = max(results_list, key=lambda x: x['MAP'])

# Print the result with the highest 'MAP'
print(max_result)

{'cutoff': 600, 'init_rec_reg_alpha': 0.1, 'init_rec_reg_beta': 0.3, 'MAP': 0.09130592834867324}


In [74]:
# Evaluate model recommendations with 5-fold CV
# NON-Normalised SIMILARITIES
# USING SPECIFIED VALUES (FIRST 5-fold CV)
alpha = 0.88
beta = 0.685907

init_rec_cutoff = 600

xgboost_model_hyperparams = {'n_estimators':50,
                             'learning_rate':1e-1,'reg_alpha':1e-1,'reg_lambda':0.3,'max_depth':5,'max_leaves':0,
                             'grow_policy':"depthwise",'objective':"pairwise",'booster':"gbtree",
                             'use_user_profile':False,'random_seed':None
                             }

folds_map = []
for k,fold in enumerate(data_folds):
  print(f'Fold {k} evaluations started')

  # Define the recommender models trained on RecommenderFold of the current fold to prepare for XGboost training
  initial_S_slim_elastic = xgboostfold_models[k]["SLIM"]
  initial_S_ibcf = xgboostfold_models[k]["IBCF"]
  initial_S_easer = xgboostfold_models[k]["EASER"]
  initial_hybrid3models_similarity = beta * ((1 - alpha) * initial_S_ibcf  + alpha * initial_S_slim_elastic) + (1 - beta) * initial_S_easer


  # Models for features generation
  initial_hybrid3models_object = ItemKNNCustomSimilarityRecommender(data_xgboostfolds[k]['train_recommender'])
  initial_hybrid3models_object.fit(initial_hybrid3models_similarity)
  initial_hybrid3models = Predictor(URM=data_xgboostfolds[k]['train_recommender'], model = initial_hybrid3models_object)

  initial_S_rp3beta = xgboostfold_models[k]["rp3beta"]
  initial_rp3beta_object = ItemKNNCustomSimilarityRecommender(data_xgboostfolds[k]['train_recommender'])
  initial_rp3beta_object.fit(initial_S_rp3beta)

  initial_S_easer = xgboostfold_models[k]["EASER"]
  initial_easer_object = ItemKNNCustomSimilarityRecommender(data_xgboostfolds[k]['train_recommender'])
  initial_easer_object.fit(initial_S_easer)

  initial_S_ibcf = xgboostfold_models[k]["IBCF"]
  initial_ibcf_object = ItemKNNCustomSimilarityRecommender(data_xgboostfolds[k]['train_recommender'])
  initial_ibcf_object.fit(initial_S_ibcf)

  init_other_algorithms = {
    "rp3beta": initial_rp3beta_object,
    "EASER": initial_easer_object,
    "IBCF":initial_ibcf_object}

  # Get XGboost trained
  xgboost_trained = train_xgboost(data_xgboostfolds[k]['train_recommender'],data_xgboostfolds[k]['train_xgboost'],
                                  initial_hybrid3models,initial_recommendations_cutoff = init_rec_cutoff, other_algorithms = init_other_algorithms,
                                  xgboost_hyperparams = xgboost_model_hyperparams)


  # Define the recommender models trained on the whole current fold to prepare for the evaluation
  S_slim_elastic = fold_models[k]["SLIM"]
  S_ibcf = fold_models[k]["IBCF"]
  S_easer = fold_models[k]["EASER"]
  hybrid3models_similarity = beta * ((1 - alpha) * S_ibcf  + alpha * S_slim_elastic) + (1 - beta) * S_easer

  hybrid3models_object = ItemKNNCustomSimilarityRecommender(fold['train'])
  hybrid3models_object.fit(hybrid3models_similarity)
  hybrid3models = Predictor(URM=fold['train'], model = hybrid3models_object)

  # Models for features generation
  S_rp3beta = fold_models[k]["rp3beta"]
  rp3beta_object = ItemKNNCustomSimilarityRecommender(fold['train'])
  rp3beta_object.fit(S_rp3beta)

  S_easer = fold_models[k]["EASER"]
  easer_object = ItemKNNCustomSimilarityRecommender(fold['train'])
  easer_object.fit(S_easer)

  S_ibcf = fold_models[k]["IBCF"]
  ibcf_object = ItemKNNCustomSimilarityRecommender(fold['train'])
  ibcf_object.fit(S_ibcf)

  other_algorithms = {
    "rp3beta": rp3beta_object,
    "EASER": easer_object,
    "IBCF":ibcf_object}

  features_df = prepare_training_df(fold['train'],fold['train'],hybrid3models,init_rec_cutoff,other_algorithms)
  features_df = features_df.drop(columns=["Label"])


  map, mp,mr = evaluate_xgboost(fold['val'],features_df,xgboost_trained)
  folds_map.append(map)


cv_map = np.array(folds_map).sum()/len(data_folds)
print(f'The crossvalidation MAP for list combination of Noramalised IBCF and SLIM elasticnet is {cv_map}')

Fold 0 evaluations started
ItemKNNCustomSimilarityRecommender: URM Detected 424 ( 3.4%) users with no interactions.
ItemKNNCustomSimilarityRecommender: URM Detected 334 ( 1.5%) items with no interactions.
ItemKNNCustomSimilarityRecommender: URM Detected 424 ( 3.4%) users with no interactions.
ItemKNNCustomSimilarityRecommender: URM Detected 334 ( 1.5%) items with no interactions.
ItemKNNCustomSimilarityRecommender: URM Detected 424 ( 3.4%) users with no interactions.
ItemKNNCustomSimilarityRecommender: URM Detected 334 ( 1.5%) items with no interactions.
ItemKNNCustomSimilarityRecommender: URM Detected 424 ( 3.4%) users with no interactions.
ItemKNNCustomSimilarityRecommender: URM Detected 334 ( 1.5%) items with no interactions.


100%|██████████| 12638/12638 [00:51<00:00, 243.05it/s]


ItemKNNCustomSimilarityRecommender: URM Detected 205 ( 1.6%) users with no interactions.
ItemKNNCustomSimilarityRecommender: URM Detected 120 ( 0.5%) items with no interactions.
ItemKNNCustomSimilarityRecommender: URM Detected 205 ( 1.6%) users with no interactions.
ItemKNNCustomSimilarityRecommender: URM Detected 120 ( 0.5%) items with no interactions.
ItemKNNCustomSimilarityRecommender: URM Detected 205 ( 1.6%) users with no interactions.
ItemKNNCustomSimilarityRecommender: URM Detected 120 ( 0.5%) items with no interactions.
ItemKNNCustomSimilarityRecommender: URM Detected 205 ( 1.6%) users with no interactions.
ItemKNNCustomSimilarityRecommender: URM Detected 120 ( 0.5%) items with no interactions.


100%|██████████| 12638/12638 [00:54<00:00, 233.09it/s]


Recommender results are: Precision = 81.2283333333, Recall = 4817.3488569762, MAP = 0.0913059283
Fold 1 evaluations started
ItemKNNCustomSimilarityRecommender: URM Detected 464 ( 3.7%) users with no interactions.
ItemKNNCustomSimilarityRecommender: URM Detected 345 ( 1.6%) items with no interactions.
ItemKNNCustomSimilarityRecommender: URM Detected 464 ( 3.7%) users with no interactions.
ItemKNNCustomSimilarityRecommender: URM Detected 345 ( 1.6%) items with no interactions.
ItemKNNCustomSimilarityRecommender: URM Detected 464 ( 3.7%) users with no interactions.
ItemKNNCustomSimilarityRecommender: URM Detected 345 ( 1.6%) items with no interactions.
ItemKNNCustomSimilarityRecommender: URM Detected 464 ( 3.7%) users with no interactions.
ItemKNNCustomSimilarityRecommender: URM Detected 345 ( 1.6%) items with no interactions.


...

Recommender results are: Precision = 81.3183333333, Recall = 4887.3573106284, MAP = 0.0921251896
Fold 4 evaluations started
ItemKNNCustomSimilarityRecommender: URM Detected 478 ( 3.8%) users with no interactions.
ItemKNNCustomSimilarityRecommender: URM Detected 337 ( 1.5%) items with no interactions.
ItemKNNCustomSimilarityRecommender: URM Detected 478 ( 3.8%) users with no interactions.
ItemKNNCustomSimilarityRecommender: URM Detected 337 ( 1.5%) items with no interactions.
ItemKNNCustomSimilarityRecommender: URM Detected 478 ( 3.8%) users with no interactions.
ItemKNNCustomSimilarityRecommender: URM Detected 337 ( 1.5%) items with no interactions.
ItemKNNCustomSimilarityRecommender: URM Detected 478 ( 3.8%) users with no interactions.
ItemKNNCustomSimilarityRecommender: URM Detected 337 ( 1.5%) items with no interactions.


100%|██████████| 12638/12638 [00:49<00:00, 253.22it/s]


ItemKNNCustomSimilarityRecommender: URM Detected 239 ( 1.9%) users with no interactions.
ItemKNNCustomSimilarityRecommender: URM Detected 124 ( 0.6%) items with no interactions.
ItemKNNCustomSimilarityRecommender: URM Detected 239 ( 1.9%) users with no interactions.
ItemKNNCustomSimilarityRecommender: URM Detected 124 ( 0.6%) items with no interactions.
ItemKNNCustomSimilarityRecommender: URM Detected 239 ( 1.9%) users with no interactions.
ItemKNNCustomSimilarityRecommender: URM Detected 124 ( 0.6%) items with no interactions.
ItemKNNCustomSimilarityRecommender: URM Detected 239 ( 1.9%) users with no interactions.
ItemKNNCustomSimilarityRecommender: URM Detected 124 ( 0.6%) items with no interactions.


100%|██████████| 12638/12638 [00:46<00:00, 270.44it/s]


Recommender results are: Precision = 80.9200000000, Recall = 4845.2773503021, MAP = 0.0913736426
The crossvalidation MAP for list combination of Noramalised IBCF and SLIM elasticnet is 0.09079961769950926


In [75]:
# Evaluate model recommendations with 5-fold CV
# NON-Normalised SIMILARITIES
# USING SPECIFIED VALUES (FIRST 5-fold CV)
alpha = 0.88
beta = 0.685907

init_rec_cutoff = 600

xgboost_model_hyperparams = {'n_estimators':50,
                             'learning_rate':1e-1,'reg_alpha':1e-1,'reg_lambda':0.1,'max_depth':5,'max_leaves':0,
                             'grow_policy':"depthwise",'objective':"pairwise",'booster':"gbtree",
                             'use_user_profile':False,'random_seed':None
                             }

folds_map = []
for k,fold in enumerate(data_folds):
  print(f'Fold {k} evaluations started')

  # Define the recommender models trained on RecommenderFold of the current fold to prepare for XGboost training
  initial_S_slim_elastic = xgboostfold_models[k]["SLIM"]
  initial_S_ibcf = xgboostfold_models[k]["IBCF"]
  initial_S_easer = xgboostfold_models[k]["EASER"]
  initial_hybrid3models_similarity = beta * ((1 - alpha) * initial_S_ibcf  + alpha * initial_S_slim_elastic) + (1 - beta) * initial_S_easer


  # Models for features generation
  initial_hybrid3models_object = ItemKNNCustomSimilarityRecommender(data_xgboostfolds[k]['train_recommender'])
  initial_hybrid3models_object.fit(initial_hybrid3models_similarity)
  initial_hybrid3models = Predictor(URM=data_xgboostfolds[k]['train_recommender'], model = initial_hybrid3models_object)

  initial_S_rp3beta = xgboostfold_models[k]["rp3beta"]
  initial_rp3beta_object = ItemKNNCustomSimilarityRecommender(data_xgboostfolds[k]['train_recommender'])
  initial_rp3beta_object.fit(initial_S_rp3beta)

  initial_S_easer = xgboostfold_models[k]["EASER"]
  initial_easer_object = ItemKNNCustomSimilarityRecommender(data_xgboostfolds[k]['train_recommender'])
  initial_easer_object.fit(initial_S_easer)

  initial_S_ibcf = xgboostfold_models[k]["IBCF"]
  initial_ibcf_object = ItemKNNCustomSimilarityRecommender(data_xgboostfolds[k]['train_recommender'])
  initial_ibcf_object.fit(initial_S_ibcf)

  init_other_algorithms = {
    "rp3beta": initial_rp3beta_object,
    "EASER": initial_easer_object,
    "IBCF":initial_ibcf_object}

  # Get XGboost trained
  xgboost_trained = train_xgboost(data_xgboostfolds[k]['train_recommender'],data_xgboostfolds[k]['train_xgboost'],
                                  initial_hybrid3models,initial_recommendations_cutoff = init_rec_cutoff, other_algorithms = init_other_algorithms,
                                  xgboost_hyperparams = xgboost_model_hyperparams)


  # Define the recommender models trained on the whole current fold to prepare for the evaluation
  S_slim_elastic = fold_models[k]["SLIM"]
  S_ibcf = fold_models[k]["IBCF"]
  S_easer = fold_models[k]["EASER"]
  hybrid3models_similarity = beta * ((1 - alpha) * S_ibcf  + alpha * S_slim_elastic) + (1 - beta) * S_easer

  hybrid3models_object = ItemKNNCustomSimilarityRecommender(fold['train'])
  hybrid3models_object.fit(hybrid3models_similarity)
  hybrid3models = Predictor(URM=fold['train'], model = hybrid3models_object)

  # Models for features generation
  S_rp3beta = fold_models[k]["rp3beta"]
  rp3beta_object = ItemKNNCustomSimilarityRecommender(fold['train'])
  rp3beta_object.fit(S_rp3beta)

  S_easer = fold_models[k]["EASER"]
  easer_object = ItemKNNCustomSimilarityRecommender(fold['train'])
  easer_object.fit(S_easer)

  S_ibcf = fold_models[k]["IBCF"]
  ibcf_object = ItemKNNCustomSimilarityRecommender(fold['train'])
  ibcf_object.fit(S_ibcf)

  other_algorithms = {
    "rp3beta": rp3beta_object,
    "EASER": easer_object,
    "IBCF":ibcf_object}

  features_df = prepare_training_df(fold['train'],fold['train'],hybrid3models,init_rec_cutoff,other_algorithms)
  features_df = features_df.drop(columns=["Label"])


  map, mp,mr = evaluate_xgboost(fold['val'],features_df,xgboost_trained)
  folds_map.append(map)


cv_map = np.array(folds_map).sum()/len(data_folds)
print(f'The crossvalidation MAP for list combination of Noramalised IBCF and SLIM elasticnet is {cv_map}')

Fold 0 evaluations started
ItemKNNCustomSimilarityRecommender: URM Detected 424 ( 3.4%) users with no interactions.
ItemKNNCustomSimilarityRecommender: URM Detected 334 ( 1.5%) items with no interactions.
ItemKNNCustomSimilarityRecommender: URM Detected 424 ( 3.4%) users with no interactions.
ItemKNNCustomSimilarityRecommender: URM Detected 334 ( 1.5%) items with no interactions.
ItemKNNCustomSimilarityRecommender: URM Detected 424 ( 3.4%) users with no interactions.
ItemKNNCustomSimilarityRecommender: URM Detected 334 ( 1.5%) items with no interactions.
ItemKNNCustomSimilarityRecommender: URM Detected 424 ( 3.4%) users with no interactions.
ItemKNNCustomSimilarityRecommender: URM Detected 334 ( 1.5%) items with no interactions.


100%|██████████| 12638/12638 [00:46<00:00, 271.55it/s]


ItemKNNCustomSimilarityRecommender: URM Detected 205 ( 1.6%) users with no interactions.
ItemKNNCustomSimilarityRecommender: URM Detected 120 ( 0.5%) items with no interactions.
ItemKNNCustomSimilarityRecommender: URM Detected 205 ( 1.6%) users with no interactions.
ItemKNNCustomSimilarityRecommender: URM Detected 120 ( 0.5%) items with no interactions.
ItemKNNCustomSimilarityRecommender: URM Detected 205 ( 1.6%) users with no interactions.
ItemKNNCustomSimilarityRecommender: URM Detected 120 ( 0.5%) items with no interactions.
ItemKNNCustomSimilarityRecommender: URM Detected 205 ( 1.6%) users with no interactions.
ItemKNNCustomSimilarityRecommender: URM Detected 120 ( 0.5%) items with no interactions.


100%|██████████| 12638/12638 [00:47<00:00, 267.92it/s]


Recommender results are: Precision = 81.2283333333, Recall = 4817.3488569762, MAP = 0.0902636282
Fold 1 evaluations started
ItemKNNCustomSimilarityRecommender: URM Detected 464 ( 3.7%) users with no interactions.
ItemKNNCustomSimilarityRecommender: URM Detected 345 ( 1.6%) items with no interactions.
ItemKNNCustomSimilarityRecommender: URM Detected 464 ( 3.7%) users with no interactions.
ItemKNNCustomSimilarityRecommender: URM Detected 345 ( 1.6%) items with no interactions.
ItemKNNCustomSimilarityRecommender: URM Detected 464 ( 3.7%) users with no interactions.
ItemKNNCustomSimilarityRecommender: URM Detected 345 ( 1.6%) items with no interactions.
ItemKNNCustomSimilarityRecommender: URM Detected 464 ( 3.7%) users with no interactions.
ItemKNNCustomSimilarityRecommender: URM Detected 345 ( 1.6%) items with no interactions.


...

Recommender results are: Precision = 81.3183333333, Recall = 4887.3573106284, MAP = 0.0918611390
Fold 4 evaluations started
ItemKNNCustomSimilarityRecommender: URM Detected 478 ( 3.8%) users with no interactions.
ItemKNNCustomSimilarityRecommender: URM Detected 337 ( 1.5%) items with no interactions.
ItemKNNCustomSimilarityRecommender: URM Detected 478 ( 3.8%) users with no interactions.
ItemKNNCustomSimilarityRecommender: URM Detected 337 ( 1.5%) items with no interactions.
ItemKNNCustomSimilarityRecommender: URM Detected 478 ( 3.8%) users with no interactions.
ItemKNNCustomSimilarityRecommender: URM Detected 337 ( 1.5%) items with no interactions.
ItemKNNCustomSimilarityRecommender: URM Detected 478 ( 3.8%) users with no interactions.
ItemKNNCustomSimilarityRecommender: URM Detected 337 ( 1.5%) items with no interactions.


100%|██████████| 12638/12638 [00:46<00:00, 269.64it/s]


ItemKNNCustomSimilarityRecommender: URM Detected 239 ( 1.9%) users with no interactions.
ItemKNNCustomSimilarityRecommender: URM Detected 124 ( 0.6%) items with no interactions.
ItemKNNCustomSimilarityRecommender: URM Detected 239 ( 1.9%) users with no interactions.
ItemKNNCustomSimilarityRecommender: URM Detected 124 ( 0.6%) items with no interactions.
ItemKNNCustomSimilarityRecommender: URM Detected 239 ( 1.9%) users with no interactions.
ItemKNNCustomSimilarityRecommender: URM Detected 124 ( 0.6%) items with no interactions.
ItemKNNCustomSimilarityRecommender: URM Detected 239 ( 1.9%) users with no interactions.
ItemKNNCustomSimilarityRecommender: URM Detected 124 ( 0.6%) items with no interactions.


100%|██████████| 12638/12638 [00:46<00:00, 270.71it/s]


Recommender results are: Precision = 80.9200000000, Recall = 4845.2773503021, MAP = 0.0916727479
The crossvalidation MAP for list combination of Noramalised IBCF and SLIM elasticnet is 0.090467533847764


# Predict for the test data

**Now let's read the data we want to predict for and see if there're overlaps with UserIDs in train data**

In [78]:
urm_pred_path = './content/data_target_users_test.csv'

urm_pred_df = pd.read_csv(filepath_or_buffer=urm_pred_path,
                                sep=",",
                                header=0,
                                dtype={0:int},
                                engine='python')

urm_pred_df.columns = ["UserID"]
len(urm_pred_df['UserID'])
print('Unique user id to predict:', urm_pred_df['UserID'].nunique())

Unique user id to predict: 10882


### Generate features and initial predictions

**Define the model to generate recommendations**

In [80]:
# Define our best submission model i.e. hybrid of NON-NORMALISED SIMILARITIES 0.685907*(0.88*SLIM elasticnet + 0.12*IBCF)+(1 - 0.685907)*EASER

alpha = 0.88
beta = 0.685907

S_slim_elastic = load_npz('./5-fold CV/Whole_data_trained_models/ItItSimilarity_trained_whole_data_noemptyprofiles_SLIMelasticnet_kaggleMAP0.13484.npz')
S_ibcf = load_npz('./5-fold CV/Whole_data_trained_models/ItItSimilarity_trained_whole_data_noemptyprofiles_IBCF_kaggleMAP0.11585.npz')
S_easer = load_npz('./5-fold CV/Whole_data_trained_models/ItItSimilarity_trained_whole_data_noemptyprofiles_EASER_kaggleMAP0.12504.npz')
hybrid_similarity = beta * ((1 - alpha) * S_ibcf_f0  + alpha * S_slim_elastic_f0) + (1 - beta) * S_easer_f0

hybrid_recommender_object = ItemKNNCustomSimilarityRecommender(urm_all)
hybrid_recommender_object.fit(hybrid_similarity)
hybrid_3models = Predictor(URM=data_fold0['train'], model = hybrid_recommender_object)



**Build the dataframe with the predictions**

In [100]:
import pandas as pd
from tqdm import tqdm
import scipy.sparse as sps
import numpy as np
from xgboost import XGBRanker

n_users = len(userID_unique)
n_items = len(itemID_unique)
n_interactions = len(urm_all_df)

feature_dataframe = pd.DataFrame(index=range(0,n_users), columns = ["ItemID"])
feature_dataframe.index.name='UserID'

In [101]:
urm_all

<13025x22348 sparse matrix of type '<class 'numpy.float64'>'
	with 478730 stored elements in Compressed Sparse Row format>

In [102]:
cutoff = 30

for user_id in userID_unique:
    recommendations = hybrid_3models.recommend(user_id, at = cutoff)
    feature_dataframe.loc[user_id, "ItemID"] = recommendations



**Transform the dataframe so that we have one recommendation per row**

In [103]:
feature_dataframe = feature_dataframe.explode("ItemID")
feature_dataframe['ItemID'] = pd.to_numeric(feature_dataframe['ItemID'])
feature_dataframe

,ItemID
UserID,
0,674
0,587
0,4
0,1449
0,1585
...,...
12637,10906
12637,327
12637,10898


**Now let's add features as the prediction of other algorithms**

In [104]:
S_rp3beta = load_npz('./5-fold CV/Whole_data_trained_models/ItItSimilarity_trained_whole_data_noemptyprofiles_RP3Beta_kaggleMAP0.09372.npz')
rp3beta_recommender_object = ItemKNNCustomSimilarityRecommender(urm_all)
rp3beta_recommender_object.fit(S_rp3beta)

S_ibcf = load_npz('./5-fold CV/Whole_data_trained_models/ItItSimilarity_trained_whole_data_noemptyprofiles_IBCF_kaggleMAP0.11585.npz')
ibcf_recommender_object = ItemKNNCustomSimilarityRecommender(urm_all)
ibcf_recommender_object.fit(S_ibcf)

S_easer = load_npz('./5-fold CV/Whole_data_trained_models/ItItSimilarity_trained_whole_data_noemptyprofiles_EASER_kaggleMAP0.12504.npz')
easer_recommender_object = ItemKNNCustomSimilarityRecommender(urm_all)
easer_recommender_object.fit(S_easer)


other_algorithms = {
    "rp3beta": rp3beta_recommender_object,
    "EASER": easer_recommender_object,
    "IBCF":ibcf_recommender_object
}


ItemKNNCustomSimilarityRecommender: URM Detected 387 ( 3.0%) users with no interactions.
ItemKNNCustomSimilarityRecommender: URM Detected 126 ( 0.6%) items with no interactions.
ItemKNNCustomSimilarityRecommender: URM Detected 387 ( 3.0%) users with no interactions.
ItemKNNCustomSimilarityRecommender: URM Detected 126 ( 0.6%) items with no interactions.
ItemKNNCustomSimilarityRecommender: URM Detected 387 ( 3.0%) users with no interactions.
ItemKNNCustomSimilarityRecommender: URM Detected 126 ( 0.6%) items with no interactions.


In [105]:
for user_id in userID_unique:
    for rec_label, rec_instance in other_algorithms.items():

        item_list = feature_dataframe.loc[user_id, "ItemID"].values.tolist()

        all_item_scores = rec_instance._compute_item_score([user_id], items_to_compute = item_list)

        feature_dataframe.loc[user_id, rec_label] = all_item_scores[0, item_list]

feature_dataframe = feature_dataframe.reset_index()
feature_dataframe = feature_dataframe.rename(columns = {"index": "UserID"})
feature_dataframe

,UserID,ItemID,rp3beta,EASER,IBCF
0,0,674,0.000000,0.000000,0.0
1,0,587,0.000000,0.000000,0.0
2,0,4,0.000000,0.000000,0.0
3,0,1449,0.000000,0.000000,0.0
4,0,1585,0.000000,0.000000,0.0
...,...,...,...,...,...
379135,12637,10906,0.062133,0.010391,0.0
379136,12637,327,0.000000,0.000000,0.0
379137,12637,10898,0.000000,0.000000,0.0
379138,12637,17581,0.000000,0.005182,0.0


**The popularity of the item, the user activity and etc. features**

In [106]:
item_popularity = np.ediff1d(sps.csc_matrix(urm_all).indptr)

feature_dataframe['item_popularity'] = item_popularity[feature_dataframe["ItemID"].values.astype(int)]
feature_dataframe

,UserID,ItemID,rp3beta,EASER,IBCF,item_popularity
0,0,674,0.000000,0.000000,0.0,129
1,0,587,0.000000,0.000000,0.0,153
2,0,4,0.000000,0.000000,0.0,1666
3,0,1449,0.000000,0.000000,0.0,52
4,0,1585,0.000000,0.000000,0.0,35
...,...,...,...,...,...,...
379135,12637,10906,0.062133,0.010391,0.0,6
379136,12637,327,0.000000,0.000000,0.0,223
379137,12637,10898,0.000000,0.000000,0.0,7
379138,12637,17581,0.000000,0.005182,0.0,6


**The profile length**

In [107]:
user_popularity = np.ediff1d(sps.csr_matrix(urm_all).indptr)

feature_dataframe['user_profile_len'] = user_popularity[feature_dataframe["UserID"].values.astype(int)]
feature_dataframe

,UserID,ItemID,rp3beta,EASER,IBCF,item_popularity,user_profile_len
0,0,674,0.000000,0.000000,0.0,129,0
1,0,587,0.000000,0.000000,0.0,153,0
2,0,4,0.000000,0.000000,0.0,1666,0
3,0,1449,0.000000,0.000000,0.0,52,0
4,0,1585,0.000000,0.000000,0.0,35,0
...,...,...,...,...,...,...,...
379135,12637,10906,0.062133,0.010391,0.0,6,11
379136,12637,327,0.000000,0.000000,0.0,223,11
379137,12637,10898,0.000000,0.000000,0.0,7,11
379138,12637,17581,0.000000,0.005182,0.0,6,11


In [108]:
feature_dataframe = feature_dataframe.sort_values("UserID").reset_index()
feature_dataframe.drop(columns = ['index'], inplace=True)
feature_dataframe

,UserID,ItemID,rp3beta,EASER,IBCF,item_popularity,user_profile_len
0,0,674,0.0,0.000000,0.0,129,0
1,0,2023,0.0,0.000000,0.0,42,0
2,0,4802,0.0,0.000000,0.0,25,0
3,0,285,0.0,0.000000,0.0,246,0
4,0,99,0.0,0.000000,0.0,477,0
...,...,...,...,...,...,...,...
379135,12637,10902,0.0,0.000000,0.0,8,11
379136,12637,16585,0.0,0.000000,0.0,3,11
379137,12637,17581,0.0,0.005182,0.0,6,11
379138,12637,17580,0.0,0.001686,0.0,1,11


**To use the ranker one first needs to specify the size of the groups, a group is the dimension you rank on, in this case each group corresponds to a user. Since we have generated a fixed number of candidate items for each user all groups have the same length.**

In [109]:
groups = feature_dataframe.groupby("UserID").size().values
groups

array([30, 30, 30, ..., 30, 30, 30], dtype=int64)

In [1]:
# Define the recommender models trained on RecommenderFold of the current fold to prepare for XGboost training
# NON-Normalised SIMILARITIES

init_rec_cutoff = 600

alpha = 0.88
beta = 0.685907

xgboost_model_hyperparams = {'n_estimators':50,
                             'learning_rate':1e-1,'reg_alpha':0.1,'reg_lambda':0.3,'max_depth':5,'max_leaves':0,
                             'grow_policy':"depthwise",'objective':"pairwise",'booster':"gbtree",
                             'use_user_profile':False,'random_seed':None
                             }


initial_S_slim_elastic = load_npz('./5-fold CV/SLIMelasticnet/ItItSimilarity_trained_fold3_noemptyprofiles_SLIMelasticnet_kaggleMAP0.13484.npz')
initial_S_ibcf = load_npz('./5-fold CV/IBCF/ItItSimilarity_trained_fold3_noemptyprofiles_IBCF_kaggleMAP0.11585.npz')
initial_S_easer = load_npz('./5-fold CV/EASER/ItItSimilarity_trained_fold3_noemptyprofiles_EASER_kaggleMAP0.12504.npz')
initial_hybrid3models_similarity = beta * ((1 - alpha) * initial_S_ibcf  + alpha * initial_S_slim_elastic) + (1 - beta) * initial_S_easer

initial_hybrid3models_object = ItemKNNCustomSimilarityRecommender(data_folds[3]['train'])
initial_hybrid3models_object.fit(initial_hybrid3models_similarity)
initial_hybrid3models = Predictor(URM=data_folds[3]['train'], model = initial_hybrid3models_object)


# Models for features generation
initial_S_rp3beta = load_npz('./5-fold CV/RP3Beta/ItItSimilarity_trained_fold3_noemptyprofiles_RP3Beta_kaggleMAP0.09372.npz')
initial_rp3beta_object = ItemKNNCustomSimilarityRecommender(data_folds[3]['train'])
initial_rp3beta_object.fit(initial_S_rp3beta)

initial_S_easer = load_npz('./5-fold CV/EASER/ItItSimilarity_trained_fold3_noemptyprofiles_EASER_kaggleMAP0.12504.npz')
initial_easer_object = ItemKNNCustomSimilarityRecommender(data_folds[3]['train'])
initial_easer_object.fit(initial_S_easer)

initial_S_ibcf = load_npz('./5-fold CV/IBCF/ItItSimilarity_trained_fold3_noemptyprofiles_IBCF_kaggleMAP0.11585.npz')
initial_ibcf_object = ItemKNNCustomSimilarityRecommender(data_folds[3]['train'])
initial_ibcf_object.fit(initial_S_ibcf)

init_other_algorithms = {
    "rp3beta": initial_rp3beta_object,
    "EASER": initial_easer_object,
    "IBCF":initial_ibcf_object}

# Get XGboost trained
xgboost_trained = train_xgboost(data_folds[3]['train'],data_folds[3]['val'],
                                  initial_hybrid3models,initial_recommendations_cutoff = init_rec_cutoff, other_algorithms = init_other_algorithms,
                                  xgboost_hyperparams = xgboost_model_hyperparams)


# Define the recommender models trained on the whole current fold to prepare for the evaluation
S_slim_elastic = load_npz('./5-fold CV/Whole_data_trained_models/ItItSimilarity_trained_whole_data_noemptyprofiles_SLIMelasticnet_kaggleMAP0.13484.npz')
S_ibcf = load_npz('./5-fold CV/Whole_data_trained_models/ItItSimilarity_trained_whole_data_noemptyprofiles_IBCF_kaggleMAP0.11585.npz')
S_easer = load_npz('./5-fold CV/Whole_data_trained_models/ItItSimilarity_trained_whole_data_noemptyprofiles_EASER_kaggleMAP0.12504.npz')
hybrid_similarity = beta * ((1 - alpha) * S_ibcf  + alpha * S_slim_elastic) + (1 - beta) * S_easer

hybrid_recommender_object = ItemKNNCustomSimilarityRecommender(urm_all)
hybrid_recommender_object.fit(hybrid_similarity)
hybrid_3models = Predictor(URM=urm_all, model = hybrid_recommender_object)

# Models for features generation
S_rp3beta = load_npz('./5-fold CV/Whole_data_trained_models/ItItSimilarity_trained_whole_data_noemptyprofiles_RP3Beta_kaggleMAP0.09372.npz')
rp3beta_object = ItemKNNCustomSimilarityRecommender(urm_all)
rp3beta_object.fit(S_rp3beta)

S_easer = load_npz('./5-fold CV/Whole_data_trained_models/ItItSimilarity_trained_whole_data_noemptyprofiles_EASER_kaggleMAP0.12504.npz')
easer_object = ItemKNNCustomSimilarityRecommender(urm_all)
easer_object.fit(S_easer)

S_ibcf = load_npz('./5-fold CV/Whole_data_trained_models/ItItSimilarity_trained_whole_data_noemptyprofiles_IBCF_kaggleMAP0.11585.npz')
ibcf_object = ItemKNNCustomSimilarityRecommender(urm_all)
ibcf_object.fit(S_ibcf)

other_algorithms = {
    "rp3beta": rp3beta_object,
    "EASER": easer_object,
    "IBCF":ibcf_object}

features_df = prepare_training_df(urm_all,urm_all,hybrid_3models,init_rec_cutoff,other_algorithms)
features_df = features_df.drop(columns=["Label"])

NameError: name 'load_npz' is not defined

In [110]:
pred_df = pd.DataFrame(columns = ['user_id','item_list'])

for userid in urm_pred_df['UserID']:
  if userid in userID_unique:
    # Map user ID to the index of model user space
    usr_idx = userid

    # Extract the predictions and features for the user
    X_to_predict = feature_dataframe[feature_dataframe["UserID"] == usr_idx]
    # Do the reranking prediction
    item_reranking_scores = xgboost_trained.predict(X_to_predict)
    if len(item_reranking_scores) == 0:
      print(f'Lenght of reranking for user {usr_idx} is 0')
    # Rerank the items according to the reranking scores
    initial_recommendations = X_to_predict.ItemID.to_list()
    reranked_recommendations = np.array(initial_recommendations)[np.argsort(item_reranking_scores)[::-1]]

    # Map item indices from model item space to original item ID
    recomendatoins = [idx for idx in reranked_recommendations][:10] # Extract only first top-10 recommendations
  else:
    # If the user isn't present in the given data, recommend TopPop items
    recomendatoins = [2, 4, 1, 7, 3, 6, 8, 9, 15, 20]

  recomendatoins = " ".join(str(item) for item in recomendatoins)
  pred_df.loc[len(pred_df)] = [userid,recomendatoins]

KeyboardInterrupt: 

In [ ]:
pred_df

,user_id,item_list
0,1,36 101 161 123 1935 403 515 699 506 52
1,2,1095 1522 12 47 656 949 50 3176 28 196
2,3,59 536 4252 648 857 956 584 259 4 2172
3,4,28 11 27 249 50 1 1719 5 136 171
4,5,1570 170 1511 77 131 8 5138 7033 9582 9039
...,...,...
10877,13020,105 6450 6198 6452 627 7395 6749 7394 1191 155
10878,13021,16115 133 269 304 6427 6722 6724 859 17945 3071
10879,13022,809 1668 1411 1446 1767 1674 1013 4688 16075 1...
10880,13023,329 299 252 119 249 1107 1124 1290 706 32


In [ ]:
pred_df.to_csv('/content/pred_best3modelsHybrid_kaggle0.13731_XGboost_reranked_Max.csv',index=False)